In [1]:
import json
import pandas as pd
from db.redis_database import RedisDatabase

In [6]:
r = RedisDatabase()
timeseries_data = r.hgetall("NSE:BHEL-EQ")

columns = ["timecode", "price", "volume"]
rows = []

for timecode, price_vol in timeseries_data.items():
    d = json.loads(price_vol)
    price = d["price"]
    vol = d["volume"]

    rows.append([timecode, price, vol])

df = pd.DataFrame(rows, columns=columns)
df.set_index("timecode", inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_values(by="timecode", inplace=True)

In [8]:
ohlc = df["price"].resample("15min").ohlc()

In [49]:
ohlc["volume_diff"] = (df["volume"].resample("1min").max()).diff()

In [16]:
a = ohlc.loc["2021-10-27 09:00:00"]
b = ohlc.loc["2021-10-27 09:15:00"]

a, b

(open     71.55
 high     71.80
 low      71.55
 close    71.80
 Name: 2021-10-27 09:00:00, dtype: float64,
 open     71.80
 high     72.85
 low      71.40
 close    72.05
 Name: 2021-10-27 09:15:00, dtype: float64)